# Install Required Libraries

In [ ]:
!pip install yt-dlp
!pip install pydub
!pip install openai
!pip install git+https://github.com/openai/whisper.git
!pip install ipywidgets
!pip install srt
!apt-get update
!apt-get install -y ffmpeg

# Import Libraries and Define Functions

In [ ]:
import yt_dlp
from pydub import AudioSegment
from openai import OpenAI
import whisper
import torch
import os
import shutil
import srt
from datetime import timedelta
from IPython.display import display, HTML
import ipywidgets as widgets

# Create directories
os.makedirs('sample_data', exist_ok=True)

def download_youtube_audio(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': 'sample_data/audio.%(ext)s',  # Save with fixed name
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    print("Downloaded YouTube audio.")

def download_drive_video(file_id):
    # Using direct download URL for publicly accessible Google Drive files
    download_url = f"https://drive.google.com/uc?id={file_id}&export=download"
    os.system(f"wget -O sample_data/video.mp4 '{download_url}'")
    print("Downloaded Google Drive video.")
    
    audio = AudioSegment.from_file('sample_data/video.mp4')
    audio.export('sample_data/audio.mp3', format='mp3')
    print("Extracted audio from video.")

def save_file(content, filename):
    with open(filename, 'w') as file:
        file.write(content)
    print(f"Saved file: {filename}")

def generate_transcript(file_path, api_key):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = whisper.load_model("base", device=device)
    result = model.transcribe(file_path)
    transcript = result['text']
    save_file(transcript, 'sample_data/Transcripts.txt')
    return transcript, result['segments']

def generate_captions(segments):
    subtitles = []
    for i, segment in enumerate(segments):
        start = timedelta(seconds=segment['start'])
        end = timedelta(seconds=segment['end'])
        content = segment['text']
        subtitles.append(srt.Subtitle(index=i, start=start, end=end, content=content))
    srt_content = srt.compose(subtitles)
    save_file(srt_content, 'sample_data/Captions.srt')
    return srt_content

def generate_youtube_description(segments):
    description_lines = []
    for segment in segments:
        start = segment['start']
        text = segment['text']
        description_lines.append(f"{start}: {text}")
    description_content = "\n".join(description_lines)
    save_file(description_content, 'sample_data/Youtube_Description.txt')
    return description_content

def generate_blog(transcript, keywords, title, api_key):
    client = OpenAI(api_key=api_key)
    prompt = f"Title: {title}\nKeywords: {keywords}\n\nTranscript:\n{transcript}\n\nPlease write an SEO-optimized blog post based on the above transcript, using the provided title and keywords. The blog post should not exceed 5000 words."
    
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=2048
        )
        blog_content = response.choices[0].message.content.strip()
        save_file(blog_content, 'sample_data/Blog.txt')
        return blog_content
    except Exception as e:
        print(f"An error occurred while generating the blog: {str(e)}")
        return None

# Create UI Elements

In [ ]:
# UI elements
api_key_input = widgets.Text(
    value='',
    placeholder='Enter your OpenAI API key',
    description='API Key:',
    disabled=False
)

url_input = widgets.Text(
    value='',
    placeholder='Enter the video URL',
    description='Video URL:',
    disabled=False
)

file_type_input = widgets.Dropdown(
    options=['youtube', 'drive'],
    value='youtube',
    description='File Type:',
    disabled=False,
)

title_input = widgets.Text(
    value='',
    placeholder='Enter the blog title',
    description='Blog Title:',
    disabled=False
)

keywords_input = widgets.Text(
    value='',
    placeholder='Enter keywords, separated by commas',
    description='Keywords:',
    disabled=False
)

submit_button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='success',
    tooltip='Submit the form',
    icon='check'
)

output = widgets.Output()

def on_submit(b):
    with output:
        output.clear_output()
        api_key = api_key_input.value
        url = url_input.value
        file_type = file_type_input.value
        keywords = keywords_input.value
        title = title_input.value
        
        # Validate inputs
        if not api_key or not url or not keywords or not title:
            print("Please fill in all fields.")
            return
        
        # Process video
        try:
            os.makedirs('sample_data', exist_ok=True)
            if file_type == 'youtube':
                download_youtube_audio(url)
                file_path = 'sample_data/audio.mp3'
            elif file_type == 'drive':
                file_id = url.split('id=')[1]
                download_drive_video(file_id)
                file_path = 'sample_data/audio.mp3'
            
            # Ensure the file exists before proceeding
            if not os.path.exists(file_path):
                print(f"Error: {file_path} does not exist.")
                return

            transcript, segments = generate_transcript(file_path, api_key)
            captions = generate_captions(segments)
            youtube_description = generate_youtube_description(segments)
            blog = generate_blog(transcript, keywords, title, api_key)

            # Display results and download links
            display(HTML(f"<h3>Transcript:</h3><pre>{transcript}</pre>"))
            display(HTML(f"<h3>Captions:</h3><pre>{captions}</pre>"))
            display(HTML(f"<h3>Youtube Description:</h3><pre>{youtube_description}</pre>"))
            display(HTML(f"<h3>Blog:</h3><pre>{blog}</pre>"))
            display(HTML(f'<a href="/mnt/data/sample_data/Transcripts.txt" download>Download Transcript</a>'))
            display(HTML(f'<a href="/mnt/data/sample_data/Captions.srt" download>Download Captions</a>'))
            display(HTML(f'<a href="/mnt/data/sample_data/Youtube_Description.txt" download>Download Youtube Description</a>'))
            display(HTML(f'<a href="/mnt/data/sample_data/Blog.txt" download>Download Blog</a>'))
        except Exception as e:
            print(f"Error processing video: {e}")

submit_button.on_click(on_submit)

# Display UI
display(api_key_input, url_input, file_type_input, title_input, keywords_input, submit_button, output)

# Clear Generated Files

In [ ]:
def clear_downloads():
    folder = 'sample_data'
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')

# Clear all downloads and generated files
clear_downloads()
print("All downloaded and generated files have been cleared.")